# ORM Quick Start

SQLAlchemy는 "파이썬을 위한 데이터베이스 툴킷"으로, 데이터베이스 연결, 쿼리 및 결과 상호 작용, SQL 문 프로그래밍 방식 구성 등을 관리하는 도구를 제공

- Core: 데이터베이스 연결 관리, SQL 쿼리 및 결과 처리, SQL 문장 프로그래밍 방식 구성 등을 위한 기본 아키텍처

- ORM (Object Relational Mapping): Core 위에 구축되어 파이썬 클래스를 데이터베이스 테이블에 매핑하고, 객체 영속성(persistence) 메커니즘인 **세션(Session)**을 제공. 이를 통해 SQL 쿼리를 파이썬 객체 관점에서 작성하고 호출 가능


## 1. 모델 선언 (Declarative Mapping)

ORM 작업을 시작하기 위한 첫 번째 단계는 데이터베이스 스키마를 파이썬 객체(모델)로 정의하는 것이다. 이를 **Declarative Mapping**이라고 부른다.

1.  **Base 클래스 정의**: 모든 ORM 모델은 `DeclarativeBase`를 상속받는 `Base` 클래스로부터 시작한다. 이 `Base` 클래스가 모델과 데이터베이스 테이블 간의 매핑을 관리하는 기반
2.  **테이블 이름 지정 (`__tablename__`)**: 각 모델 클래스는 매핑될 데이터베이스 테이블의 이름을 `__tablename__`이라는 특별한 클래스 속성을 통해 지정해야 한다. **이 속성을 빠뜨리면 안 됨!**
3.  **컬럼 선언 (`Mapped`, `mapped_column`)**:
      * 테이블의 컬럼들은 `Mapped` **타입 어노테이션**을 사용하여 정의한다. 예를 들어, `id: Mapped[int]`와 같이 선언하면 파이썬 타입(`int`)이 자동으로 해당 SQL 타입(`INTEGER`)으로 변환된다. `Mapped[Optional[str]]`와 같이 `Optional`을 사용하면 해당 컬럼이 NULL 값을 허용함을 나타낸다.
      * `mapped_column()` 지시문은 컬럼에 대한 더 세부적인 설정을 할 때 사용한다. 예를 들어, `String(30)`처럼 SQL 특정 타입을 지정하거나, `primary_key=True`로 **기본 키**를 지정하고, `ForeignKey`를 사용하여 **외래 키** 제약 조건을 설정할 수 있다. **모든 ORM 매핑 클래스에는 최소 하나의 기본 키 컬럼이 있어야 한다.**
      * **관계 설정 (`relationship()`)**: `relationship()`은 두 ORM 클래스(모델) 간의 논리적인 연결을 나타내는 데 사용된다. 이를 통해 데이터베이스 테이블 간의 관계(예: 일대다, 다대다)를 파이썬 객체 수준에서 정의할 수 있다.
        ```python
        from typing import List, Optional
        from sqlalchemy import String
        from sqlalchemy.orm import Mapped, mapped_column, relationship
        from sqlalchemy.ext.declarative import declarative_base

        Base = declarative_base() # Base 클래스 정의

        class User(Base):
            __tablename__ = "user_account" # 테이블 이름 지정

            id: Mapped[int] = mapped_column(primary_key=True) # 기본 키
            name: Mapped[str] = mapped_column(String(30))
            fullname: Mapped[Optional[str]] # NULL 허용 컬럼

            # User와 Address 간의 일대다 관계 설정
            addresses: Mapped[List["Address"]] = relationship(
                back_populates="user", # 양방향 관계 설정
                cascade="all, delete-orphan" # 부모 삭제 시 자식도 삭제
            )

            def __repr__(self) -> str:
                return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"
        ```
          * 위 예시에서 `addresses: Mapped[List["Address"]]`는 한 명의 `User`가 여러 개의 `Address`를 가질 수 있음을 나타내는 타입 힌트이다.
          * `back_populates="user"`는 `User` 객체의 `addresses`와 `Address` 객체의 `user` 속성을 서로 연결하여 **양방향 관계**를 설정한다. 이를 통해 한쪽에서 객체를 추가하거나 삭제할 때 다른 쪽에서도 자동으로 동기화된다.
          * `cascade="all, delete-orphan"`은 부모 객체(`User`)가 삭제될 때 연결된 자식 객체(`Address`)도 함께 삭제되도록 설정하는 옵션이다.
      * **`__repr__()` 메서드**: 필수는 아니지만, 이 메서드를 정의하면 객체를 출력할 때 디버깅에 매우 유용하다. 객체의 주요 속성들을 포함하여 객체를 쉽게 알아볼 수 있는 문자열을 반환하도록 한다.

-----

## 2. 엔진 생성 (create_engine)

**Engine**은 데이터베이스 연결을 생성하는 팩토리 역할을 한다. 실제 데이터베이스 연결은 이 Engine을 통해 이루어지며, 연결 풀을 관리하여 연결을 빠르게 재사용할 수 있게 해준다.

```python
from sqlalchemy import create_engine

# SQLite 인메모리 데이터베이스 엔진 생성, echo=True로 SQL 쿼리 로그 출력
engine = create_engine("sqlite://", echo=True)
```

`create_engine("sqlite://", echo=True)`와 같이 사용하며, `echo=True`는 데이터베이스 연결에서 발생하는 모든 SQL 쿼리를 표준 출력으로 로그하도록 지시한다. 학습 및 디버깅 단계에서 매우 유용하다.

-----

## 3. DDL(Data Definition Language) 생성 (MetaData.create_all)

정의된 모델(테이블 메타데이터)과 생성된 엔진을 사용하여 대상 데이터베이스에 실제 스키마(테이블)를 생성할 수 있다.

```python
# Base에 정의된 모든 테이블을 데이터베이스에 생성
Base.metadata.create_all(engine)
```

`Base.metadata.create_all(engine)` 메서드를 호출하면 `Base` 클래스에 정의된 모든 ORM 모델에 해당하는 테이블이 데이터베이스에 생성된다.

-----

## 4. 객체 생성 및 영속화 (Session)

데이터베이스에 데이터를 삽입하려면, 정의한 모델 클래스의 인스턴스를 생성하고 **세션(Session)** 객체를 사용하여 데이터베이스에 전달해야 한다.

  * **Session**: 데이터베이스와 상호작용하며, ORM 객체의 변경 사항을 **추적**하고 트랜잭션을 관리하는 핵심 객체이다.
  * **`with Session(engine) as session:`**: 세션을 **컨텍스트 관리자** 스타일로 사용하는 것이 강력히 권장된다. 이 방식은 작업을 완료하면 세션이 자동으로 닫히고 데이터베이스 리소스가 안전하게 해제되도록 보장한다.
  * **`session.add_all()`**: 여러 객체를 한 번에 세션에 추가한다.
  * **`session.commit()`**: 세션에 보류 중인 모든 변경 사항(예: 새로운 객체 추가, 객체 속성 변경)을 데이터베이스에 **플러시(flush)*하고 현재 데이터베이스 트랜잭션을 커밋하여 변경 사항을 영구적으로 저장한다.


```python
from sqlalchemy.orm import Session

# 새로운 User 객체 생성
user1 = User(name="spongebob", fullname="Spongebob Squarepants")
user2 = User(name="sandy", fullname="Sandy Cheeks")

# 세션을 통해 데이터베이스에 객체 추가 및 커밋
with Session(engine) as session:
    session.add_all([user1, user2]) # 여러 객체 한 번에 추가
    session.commit() # 변경 사항 커밋
    print(f"Added users: {user1.id}, {user2.id}") # 커밋 후 ID 할당 확인 가능
```

-----

## 5 데이터 조회 (select, Session.scalars, where, join, Session.get)

데이터베이스에 저장된 데이터를 가져오는 것은 ORM의 핵심 기능 중 하나이다.

  * **`select()`**: SQL `SELECT` 문을 생성하는 함수이다.
  * **`Session.scalars()`**: `select()`로 생성된 쿼리를 실행하고, 선택된 ORM 객체들을 반복하는 `ScalarResult` 객체를 반환한다. 단일 컬럼을 조회할 때 유용하다.
  * **`Select.where()`**: 쿼리에 `WHERE` 조건을 추가하여 특정 조건을 만족하는 데이터만 가져오도록 한다.
  * **`Select.join()`**: 여러 테이블을 `JOIN`하여 쿼리할 때 사용한다. 관계가 설정된 모델 간에 데이터를 효율적으로 가져올 수 있다.
  * **`Session.get(Model, primary_key)`**: 특정 모델과 기본 키를 사용하여 단일 객체를 효율적으로 가져올 때 사용한다. 이 방법은 기본 키를 통한 조회를 위해 최적화되어 있다.


```python
from sqlalchemy import select

with Session(engine) as session:
    # 모든 User 객체 조회
    stmt = select(User)
    for user in session.scalars(stmt):
        print(f"Found user: {user}")

    # 특정 이름을 가진 User 객체 조회
    stmt = select(User).where(User.name == "spongebob")
    spongebob = session.scalars(stmt).first() # 첫 번째 결과만 가져옴
    print(f"Specific user: {spongebob}")

    # ID를 통해 User 객체 조회 (가장 효율적)
    user_by_id = session.get(User, 1) # User 모델의 기본 키가 1인 객체 조회
    print(f"User by ID: {user_by_id}")
```

-----

## 6 데이터 변경 (commit, flush)

세션은 ORM 매핑된 객체의 변경 사항을 자동으로 **추적**한다. 객체의 속성을 변경한 후 `session.commit()`을 호출하면 SQL `UPDATE` 문이 실행되어 변경 사항이 데이터베이스에 반영된다.

  * **`session.flush()`**: 트랜잭션을 커밋하지 않고, 보류 중인 변경 사항에 대한 SQL(예: `INSERT`, `UPDATE`, `DELETE`)을 데이터베이스에 즉시 전송한다. 이는 변경 사항을 데이터베이스에 반영하지만, 아직 트랜잭션이 완료되지 않아 롤백이 가능한 상태로 유지된다. `session.commit()` 전에 ID 값을 미리 얻거나 다른 작업을 수행해야 할 때 유용하다.


```python
with Session(engine) as session:
    # ID가 1인 User 객체 조회
    user_to_update = session.get(User, 1)
    if user_to_update:
        user_to_update.fullname = "Squidward Tentacles" # 객체 속성 변경
        session.commit() # 변경 사항 커밋
        print(f"Updated user: {user_to_update}")
```

-----

## 7 데이터 삭제 (remove, Session.delete)

데이터베이스에서 객체를 삭제하는 방법은 크게 두 가지이다.

  * **`relationship`을 통한 삭제 (`collection.remove()`):** `relationship()`에 `cascade` 옵션이 적절히 설정되어 있다면, 관계가 설정된 컬렉션에서 객체를 제거하는 방식으로 해당 객체를 데이터베이스에서 삭제할 수 있다. 예를 들어, 부모 객체에서 자식 객체를 컬렉션에서 제거하면, `delete-orphan`과 같은 cascade 설정에 따라 자식 객체도 삭제될 수 있다.
  * **`session.delete()`**: 최상위 객체를 삭제할 때 사용한다. 이 메서드는 객체를 다음 플러시(flush) 시 삭제되도록 설정한다. 이 역시 `cascade` 옵션에 따라 관련된 객체도 함께 삭제될 수 있다.


```python
with Session(engine) as session:
    # ID가 2인 User 객체 조회 후 삭제
    user_to_delete = session.get(User, 2)
    if user_to_delete:
        session.delete(user_to_delete) # 객체를 삭제하도록 세션에 표시
        session.commit() # 삭제 작업 커밋
        print(f"Deleted user: {user_to_delete}")

    # 삭제 확인
    deleted_user = session.get(User, 2)
    print(f"User after deletion attempt: {deleted_user}") # None이 출력될 것임
```

-----


---


훈련 과제 1: 나만의 모델 만들기 다음 요구사항에 맞춰 파이썬 코드로 두 개의 ORM 모델을 선언
1. Product 모델:

테이블 이름은 "products"로 지정.

- id: 정수형 기본 키 (자동 증가).

- name: 255자 길이의 문자열, NULL 불가.

- price: 실수형, 기본값은 0.0.

- category_id: 정수형, Category 테이블의 id를 참조하는 외래 키.

- __repr__ 메서드를 포함하여 객체 정보를 쉽게 확인할 수 있도록 할 것.
2. Category 모델:

- 테이블 이름은 "categories"로 지정.

- id: 정수형 기본 키 (자동 증가).

- name: 100자 길이의 문자열, NULL 불가, 고유(unique)해야 함.

- products: Product 모델과의 일대다 관계를 설정할 것. back_populates와 cascade를 적절히 사용해라.

```python
# 필요한 모듈 임포트
# from typing import List, Optional
# from sqlalchemy import ForeignKey, String, Integer, Float
# from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship

# Base 클래스 정의
# class Base(DeclarativeBase):
#     pass

# Product 및 Category 모델 정의
# ... 너의 코드를 여기에 작성해봐!
```

In [ ]:
# 필요한 모듈 임포트
from typing import List, Optional
from sqlalchemy import ForeignKey, String, Integer, Float
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship

# Base 클래스 정의
class Base(DeclarativeBase):
    pass

# Product 및 Category 모델 정의
class Product(Base):
    __tablename__ = "products"
    
    i: Mapped[int] = mapped_column(primary_key=True, autoincrement=True)
    name: Mapped[str] = mapped_column(String(30))
    price: Mapped[float] = mapped_column(default=0.0)
    category_id: Mapped[int] = mapped_column(ForeignKey("categories.id")) # fk
    category: Mapped["Category"] = relationship(back_populates="products")

    def __repr__(self) -> str:
        return f"Prodict(id={self.id!r}, )"


class Category(Base):
    __tablename__ = "categories"
    id: Mapped[int] = mapped_column(primary_key=True, autoincrement=True)
    name: Mapped[str] = mapped_column(String(100), unique=True)
    products: Mapped[List["Product"]] = relationship(back_populates="category",
                                                     cascade="all, delete-orphan" )
    def __repr__(self) -> str: 
        return f"Category(id={self.id!r}), name={self.name!r}"


        

Product 테이블이 성공적으로 생성되었습니다!
<Product(id=1, name='노트북', price=1200000)>
